In [141]:
import os
import json
import pandas as pd 
import traceback
from dotenv import load_dotenv

In [142]:
from langchain_openai import AzureChatOpenAI

In [143]:
load_dotenv()

True

In [144]:
llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_DEPLOYMENT_NAME"),
    api_version=os.getenv('API_VERSION'),
    temperature=0.6,
    max_tokens=None,
    timeout=None,
    api_key=os.getenv('API_KEY'),
    azure_endpoint=os.getenv('AZURE_ENDPOINT')
)

In [145]:
llm.invoke("Tell me a joke")

AIMessage(content="Why don't skeletons fight each other?\n\nThey don't have the guts.", response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 11, 'total_tokens': 26}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_6d6abe0201', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-7775ed43-4e39-46dc-ad7e-32879efdd71d-0', usage_metadata={'input_tokens': 11, 'output_tokens': 15, 'total_tokens': 26})

In [146]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback


In [147]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [148]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide, and the response returned my be just json string, dont not any extra symbol it \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [149]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [150]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [151]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [152]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [153]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [154]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [155]:
file_path = "/Users/a/Desktop/Machine Learning/data.txt"

In [156]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [157]:
TEXT

'Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]\n\nData science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]\n\nData science is a "concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] However, data science is different from computer science and information science. Turing Award winner Jim Gray

In [158]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [159]:
NUMBER=10
SUBJECT="data science"
TONE="simple"

In [160]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]

Data science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]

Data science is a "concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] However

In [161]:
response

{'text': 'Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]\n\nData science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]\n\nData science is a "concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] However, data science is different from computer science and information science. Turing Award winner

In [162]:
type(response)

dict

In [163]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")
print(f"Total Successful request:{cb.successful_requests}")

Total Tokens:2700
Prompt Tokens:1448
Completion Tokens:1252
Total Cost:0.11856
Total Successful request:2


In [164]:
quiz_str=response.get("quiz")

In [165]:
quiz_dict = json.loads(quiz_str)

In [166]:
quiz_dict

{'1': {'mcq': 'What does data science use to extract knowledge from data?',
  'options': {'a': 'Only statistics',
   'b': 'Only algorithms',
   'c': 'Statistics, scientific computing, and algorithms',
   'd': 'Only domain knowledge'},
  'correct': 'c'},
 '2': {'mcq': 'Data science integrates domain knowledge from?',
  'options': {'a': 'Only natural sciences',
   'b': 'Natural sciences, information technology, and medicine',
   'c': 'Only information technology',
   'd': 'Only medicine'},
  'correct': 'b'},
 '3': {'mcq': 'How can data science be described?',
  'options': {'a': 'Only as a science',
   'b': 'Only as a discipline',
   'c': 'Only as a workflow',
   'd': 'As a science, a research paradigm, a research method, a discipline, a workflow, and a profession'},
  'correct': 'd'},
 '4': {'mcq': 'What is data science a concept to unify?',
  'options': {'a': 'Statistics and data analysis',
   'b': 'Informatics and data analysis',
   'c': 'Statistics, data analysis, informatics, and rel

In [167]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [168]:
quiz_table_data

[{'MCQ': 'What does data science use to extract knowledge from data?',
  'Choices': 'a: Only statistics | b: Only algorithms | c: Statistics, scientific computing, and algorithms | d: Only domain knowledge',
  'Correct': 'c'},
 {'MCQ': 'Data science integrates domain knowledge from?',
  'Choices': 'a: Only natural sciences | b: Natural sciences, information technology, and medicine | c: Only information technology | d: Only medicine',
  'Correct': 'b'},
 {'MCQ': 'How can data science be described?',
  'Choices': 'a: Only as a science | b: Only as a discipline | c: Only as a workflow | d: As a science, a research paradigm, a research method, a discipline, a workflow, and a profession',
  'Correct': 'd'},
 {'MCQ': 'What is data science a concept to unify?',
  'Choices': 'a: Statistics and data analysis | b: Informatics and data analysis | c: Statistics, data analysis, informatics, and related methods | d: Only statistics and informatics',
  'Correct': 'c'},
 {'MCQ': 'What fields does dat

In [169]:
df=pd.DataFrame(quiz_table_data)

In [170]:
df

,MCQ,Choices,Correct
0,What does data science use to extract knowledg...,a: Only statistics | b: Only algorithms | c: S...,c
1,Data science integrates domain knowledge from?,a: Only natural sciences | b: Natural sciences...,b
2,How can data science be described?,a: Only as a science | b: Only as a discipline...,d
3,What is data science a concept to unify?,a: Statistics and data analysis | b: Informati...,c
4,What fields does data science use techniques a...,a: Only mathematics | b: Only statistics | c: ...,d
5,"Who imagined data science as a ""fourth paradig...",a: Einstein | b: Jim Gray | c: Newton | d: Turing,b
6,What does a data scientist do?,a: Only creates programming code | b: Only ana...,c
7,What skills does data science incorporate?,a: Only computer science | b: Only statistics ...,d
8,According to the American Statistical Associat...,a: Database management | b: Statistics and mac...,d
9,Data science aims to solve problems in?,a: Only technology domains | b: Only natural s...,c


In [171]:
df.to_csv("Data_Science_Quiz.csv",index=False)